In [2]:
import logging ;logger = logging.getLogger(__name__); logging.basicConfig(level='INFO'); load('main.sage')

skmod = QuantumAPolynomial('4_1')
skmod.compute_skein_module()

new_basis = [skmod.gens_dict['qrt2'],skmod.meridian,skmod.longitude]

for vec in skmod.invariant_sublattice.basis():
    if vec not in span(new_basis):
        new_basis.append(vec)

inv_lattice = skmod.invariant_sublattice.submodule_with_basis(new_basis)

restricted_omega = inv_lattice.basis_matrix()*skmod.omega_with_q*(inv_lattice.basis_matrix().transpose())

doubled_res_omega = Matrix.block([[restricted_omega[1:,1:],-restricted_omega[1:,1:]],[-restricted_omega[1:,1:],restricted_omega[1:,1:]]])

INFO:__main__:A-polynomial for 4_1:	M^8*L - M^6*L + M^4*L^2 - 2*M^4*L + M^4 - M^2*L + L
INFO:__main__:After changing L to -L our A-polynomial is divisible by the reference one!


Defining qrt2, M, L, w0, w0i


Vector space of dimension 61 over Rational Field

In [55]:
center = skmod.omega_with_q.right_kernel()
central_relations = []
for rel in skmod.monomial_relations:
    if center.intersection(center.ambient_module().submodule([rel])).dimension() == 1:
        central_relations.append(rel)

In [59]:
relations_submod = center.ambient_module().submodule(skmod.monomial_relations)
center.intersection(relations_submod)

Vector space of degree 61 and dimension 7 over Rational Field
Basis matrix:
7 x 61 dense matrix over Rational Field

# Set up g_algebra

In [496]:
S = FractionField(QQ['q'])
q = S.gen()
#A.<x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36> = FreeAlgebra(S,36)

dim_wo_q = inv_lattice.dimension()-1
A2 = FreeAlgebra(S,2*dim_wo_q,['M','L'] + ['x{}'.format(i) for i in range(1,dim_wo_q-2+1)]+['Mi','Li']+['xi{}'.format(i) for i in range(1,dim_wo_q-2+1)])
FA_gens_dict = {A2.gen(i) : [0]*(i) + [1] + [0]*(2*dim_wo_q-i-1) for i in range(A2.ngens())}

Free Algebra on 72 generators (M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34) over Fraction Field of Univariate Polynomial Ring in q over Rational Field

In [497]:
import itertools
relations_FA = {}
for g2,g1 in itertools.combinations(FA_gens_dict.keys(),2):
    #print(matrix(x_gens_dict[g1])*(restricted_omega*(matrix(x_gens_dict[g2]).transpose())))
    power = (matrix(FA_gens_dict[g2])*doubled_res_omega*(matrix(FA_gens_dict[g1]).transpose()))[0][0]
    if power != 0:
        this_rel = [g1*g2,q**int(power)*g2*g1]
        #print(this_rel)
        #this_rel.sort(reverse=True)
        relations_FA[this_rel[0]] = this_rel[1]
        

H2 = A2.g_algebra(relations_FA)
H2.inject_variables()

Defining M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34


In [738]:
# Functions to get the skein algebra name for an element of the g_algebra

def get_folded_coordinate(coord):
    """
    Assumes the first coordinate is q, which can be negative and shouldn't be doubled
    ex:
     ( 1,0,2,1,0) -> ( 1,-1,2)
     (-1,0,2,3,0) -> (-1,-3,2)
    """
    l = len(coord)
    l_coord = list(coord)
    return vector(l_coord[:(l+1)/2]) - vector([0]+l_coord[(l+1)/2:])

def get_coord_from_g_alg(element,gens_dict=FA_gens_dict,fold=True):
    """
    Changes a monomial in a g_algebra to a coordinate."""
    q_pow_lambda = lambda s : '-'+s.split('^')[-1].strip(')') if s.startswith('1/') else s.split('^')[-1].strip(')')
    x_pow_lambda = lambda x : 1 if x.count('^') == 0 else int(x.split('^')[-1].strip(')'))
                                                                                                                       
    FA = element.parent().free_algebra()
    if len(element.monomials()) > 1:
        print("Only monomials have coordinates!")
        return None
    
    str_list = str(element).split('*')
    x_vars = [vector([0]+list(x_pow_lambda(v)*vector(FA_gens_dict[FA(v.split('^')[0])]))) for v in str_list if v[0] == 'x']
    # 1/q should be replaced by 1, since we add a - when there's a 1/ in the string
    scalars = [vector([int(q_pow_lambda(v).replace('1/q','1').replace('q','1').replace('--',''))] + [0]*FA.ngens()) for v in str_list if v[0] != 'x']
    return get_folded_coordinate(reduce(FA.summation,scalars+x_vars)) if fold else vector(reduce(FA.summation,scalars+x_vars))
    
def get_skalg_name_from_g_alg(element,gens_dict):
    return QuantumAPolynomial.lattice_coord_to_dict(get_coord_from_g_alg(element),gens_dict)

In [702]:
a_elem = q*A2('x1')*A2('x2')
s_elem = 1/q
s_elem_in_A2 = (s_elem*A2('1'))
dir(a_elem)

['__add__',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getmetaclass__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pari__',
 '__pos__',
 '__pow__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_act_on_',
 '_acted_upon_',
 '_add_',
 '_add_parent',
 '_ascii_art_',
 '_axiom_',
 '_axiom_init_',
 '_cache_key',
 '_cmp_',
 '_coeff_repr',
 '

In [736]:
get_folded_coordinate([1,0,1,1,0])
elm = 1/q^2*x1 + x2
mon = 1/q^2*x1


In [259]:
# little check: the commutation relation agrees on M and L:
print(matrix(skmod.longitude)*skmod.omega_with_q*(matrix(skmod.meridian).transpose()))
print(M*L == L*M)

[-1]
True


## old g_algebra setup - without inverses

In [ ]:
x_gens = [v for k,v in A.gens_dict().items() if k[0] == 'x']
x_gens_dict = {x_gens[i]: [0]*(i+1) + [1] + [0]*(37-i-2) for i in range(len(x_gens))}

In [ ]:
import itertools
relations = {}
for g2,g1 in itertools.combinations(x_gens,2):
    #print(matrix(x_gens_dict[g1])*(restricted_omega*(matrix(x_gens_dict[g2]).transpose())))
    pow = (matrix(x_gens_dict[g1])*restricted_omega*(matrix(x_gens_dict[g2]).transpose()))[0][0]
    if pow != 0:
        this_rel = [g1*g2,q**int(pow)*g2*g1]
        #this_rel.sort(reverse=True)
        relations[this_rel[0]] = this_rel[1]
        
    
H = A.g_algebra(relations)



In [ ]:
H.inject_variables()

In [ ]:
relations

# Get relations into g_algebra

In [12]:
def get_doubled_lattice_coord(coord):
    """
    Assumes the first coordinate is q, which can be negative and shouldn't be doubled
    ex:
    (1,-1,2) ->  ( 1,0,2,1,0)
    (-1,-3,2) -> (-1,0,2,3,0)
    """
    
    pos = vector([max(coord[i],0) for i in range(len(coord))])
    neg = vector([min(coord[i],0) for i in range(len(coord))])
    return list(pos) + list(-1*neg[1:])

get_doubled_lattice_coord(skmod.monomial_relations[1])

def get_g_alg_element_from_coord(coord,g_alg,qq_overide_value=False):
    """
    Assumes the coordinate is for the skein algebra, i.e. has q as the first coordinate and needs to be doubled to deal with inverses
    
    Parameters:
    coord vector
    g_alg a_algebra
    qq (default: False) - if this is set then override the default value for qq
    """
    doubled_coord = get_doubled_lattice_coord(coord)
    #logger.debug("coord: {}".format(coord))
    #logger.debug("doubled_coord: {}".format(doubled_coord))
    #logger.debug("ngens: {0}, double_coord length:{1}".format(g_alg.ngens(),len(doubled_coord)))
    if qq_overide_value:
        qq = qq_overide_value
    else:
        qq = g_alg.base().gen()
        
    free_alg = g_alg.free_algebra()
    # make the monomial as a string:
    without_q = '*'.join([
        '{0}^({1})'.format(g_alg.gen(i),doubled_coord[i])
        for i in range(1,len(doubled_coord)-1) if doubled_coord[i] != 0
    ])
    return qq^doubled_coord[0]*g_alg(free_alg(without_q))

get_g_alg_element_from_coord(inv_lattice.basis_matrix()*skmod.monomial_relations[1],H2)

q*x1^2*x3^2*x7^2*x10*x12^4*x13*x17*x18^3*x20^2*x23^2*x27^2*x31^2*x34^6*xi2^2*xi6^2*xi8*xi11^5*xi16*xi21^2*xi24^2*xi28^2*xi32^2

In [748]:
monomial_relations = [get_g_alg_element_from_coord(inv_lattice.basis_matrix()*c,H2)-1 for c in skmod.monomial_relations]
crossing_rel = [
    reduce(H2.summation,
           [-1]+[
               get_g_alg_element_from_coord(inv_lattice.basis_matrix()*mon,H2)
               for mon in relation]
          ) for relation in skmod.crossing_relations]
inverses_relations = [M*Mi-1,L*Li-1]+[H2(A2('x{0}*xi{1}'.format(j,j)))-1 for j in range(1,dim_wo_q-2+1)]

relations_ideal = H2.ideal(monomial_relations + crossing_rel+inverses_relations)

## Debug full elimination ideal.

In [765]:
all_sus_variables = []
all_maybe_fine_variables = []

for i in range(len(monomial_relations)):
    small_elimination_ideal = H2.ideal(monomial_relations[i:i+2]).elimination_ideal([g for name,g in H2.gens_dict().items() if name[0]=='x'])
    if not small_elimination_ideal.is_zero():
        #print(small_elimination_ideal.gens(), monomial_relations[i:i+2])
        for rel in monomial_relations[i:i+2]:
            for mon in rel.monomials():
                all_sus_variables += A2(str(mon)).variables()
        
    else:
        for rel in monomial_relations[i:i+2]:
            for mon in rel.monomials():
                all_maybe_fine_variables += A2(str(mon)).variables()
        #print("ZERO:  ",monomial_relations[i:i+2])

sus_set = set(all_sus_variables)
maybe_okay_set = set(all_maybe_fine_variables)
print("sus: ",sus_set)
print("maybe okay: ", maybe_okay_set)
print("intersection: ", sus_set.intersection(maybe_okay_set))
print("only in okay: ",maybe_okay_set.difference(sus_set))
print("only in sus: ",sus_set.difference(maybe_okay_set))

sus:  {x29, x31, xi20, xi31, xi22, x12, xi1, L, x22, xi23, xi29, x10, x18, x9, x27, x11, x34, xi15, x5, x8, Li, xi3, xi7, x23, xi9, xi16, x17, xi21, x4, xi32, x15, x6, x21, xi5, x32, x25, xi19, x28, xi28, xi13, xi6, x30, xi26, x16, xi30, xi34, xi33, x13, xi24, xi2, x1, x2, x26, xi8, x24, x19, xi4, Mi, xi18, x14, x3, x20, xi11, xi25, x33, xi17, x7, xi14}
maybe okay:  {x29, xi20, xi29, x9, x10, x11, x27, Li, xi15, x8, x5, xi7, x23, xi9, x4, xi19, x6, x21, x28, xi13, xi30, xi33, x13, xi24, xi2, x2, x7, x26, xi8, xi4, Mi, x3, xi25, xi11, x33, xi1, xi14}
intersection:  {x29, xi20, xi1, xi29, x10, x9, x27, x11, Li, x5, xi15, x8, xi7, x23, xi9, x4, xi19, x6, x21, x28, xi13, xi30, xi33, x13, xi24, xi2, x2, x26, xi8, xi4, Mi, x3, xi25, xi11, x33, x7, xi14}
only in okay:  set()
only in sus:  {x31, xi31, xi22, x12, L, x22, x18, xi23, xi3, x34, xi16, x17, xi21, xi32, x15, xi5, x32, x25, xi28, xi6, x30, xi26, x16, xi34, x1, x24, x19, xi18, x14, x20, xi17}


In [100]:
zero_example = H2.ideal([q*x2*x3*x4 - 1, q*x5*x6*x7*xi1 - 1])
all_example = H2.ideal([x9*x19^2*x33*xi8*xi30 - 1, q*x2*x3*x4 - 1])

In [147]:
all_x = [g for name,g in H2.gens_dict().items() if name[0]=='x']
print("eliminate x variables: ", zero_example.elimination_ideal(all_x).gens())
print("eliminate x variables:",all_example.elimination_ideal(all_x).gens())


print("Groebner basis: ", zero_example.std().gens())
print("Groebner basis: ", all_example.std().gens())

eliminate x variables:  (0,)
eliminate x variables: (1,)
Groebner basis:  (q*x2*x3*x4 - 1, q*x5*x6*x7*xi1 - 1)
Groebner basis:  (1,)


In [316]:
# small example - bad case
sus_variables = [x9, x19, x33, xi8, xi30, x2, x3, x4]
orig_relations = {}

import itertools
split_vars = lambda monomial : [A2(f) for f in str(monomial.lm()).split('*')]

for g2,g1 in itertools.combinations(sus_variables,2):
    if g1*g2 != g2*g1:
        variables = split_vars(g1*g2)
        variables.sort(reverse=True)
        key_rel = A2.product(*variables)
        orig_relations[key_rel] = H2.relations().get(key_rel)
        
        
        


--


Free Algebra on 72 generators (M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34) over Fraction Field of Univariate Polynomial Ring in q over Rational Field

In [744]:
# Try different relations.
#TODO - this doesn't let me play with things we assume commute. I'm pretty confident about those on the level of the skein algebra, but here the rules are different.
test_rel = {
    A2('xi30')*A2('x9'): 1/q*A2('x9')*A2('xi30'), # OKAY
    A2('xi8')*A2('x19'): q*A2('x19')*A2('xi8'), # OKAY
    A2('xi30')*A2('x19'): q*A2('x19')*A2('xi30'), # OKAY
    A2('x19')*A2('x2'): A2('x2')*A2('x19'), # this one breaks it! only q^0 works for q^-10 to q^10
    A2('x19')*A2('x4'): A2('x4')*A2('x19'), # this one breaks it! only q^0 works for q^-10 to q^10
    A2('xi30')*A2('x4'): A2('x4')*A2('xi30'), # this one breaks it! only q^0 works for q^-10 to q^10
    A2('x3')*A2('x2'): 1/q*A2('x2')*A2('x3') # OKAY
}

test_rel_full = H2.relations()
test_rel_full.update(test_rel)
H2_test = A2.g_algebra(test_rel)

H2_test.inject_variables()
monomial_relations_test = [get_g_alg_element_from_coord(inv_lattice.basis_matrix()*c,H2_test)-1 for c in skmod.monomial_relations]
crossing_rel_test = [
    reduce(H2_test.summation,
           [-1]+[
               get_g_alg_element_from_coord(inv_lattice.basis_matrix()*mon,H2_test)
               for mon in relation]
          ) for relation in skmod.crossing_relations]
inverses_relations_test = [H2_test(A2('M*Mi'))-1,H2_test(A2('L*Li'))-1]+[H2_test(A2('x{0}*xi{1}'.format(j,j)))-1 for j in range(1,dim_wo_q-2+1)]

relations_ideal_test = H2_test.ideal(monomial_relations_test + crossing_rel_test+inverses_relations_test)

# test with smaller ideal
bad_ideal_test = H2_test.ideal([H2_test(A2('x9*x19^2*x33*xi8*xi30')) - 1, q*H2_test(A2('x2*x3*x4')) - 1])
bad_ideal_test.std()

Defining M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34


Left Ideal (q*x2*x3*x4 - 1, x9*x19^2*x33*xi8*xi30 - 1) of Noncommutative Multivariate Polynomial Ring in M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34 over Fraction Field of Univariate Polynomial Ring in q over Rational Field, nc-relations: {x3*x2: 1/q*x2*x3,
 xi30*x9: 1/q*x9*xi30,
 xi8*x19: q*x19*xi8,
 xi30*x19: q*x19*xi30}

In [741]:
print('x2: ',get_skalg_name_from_g_alg(x2,skmod.gens_dict))
print('x4: ',get_skalg_name_from_g_alg(x4,skmod.gens_dict))
print('x19: ',get_skalg_name_from_g_alg(x19,skmod.gens_dict))
print('xi30: ',get_skalg_name_from_g_alg(xi30,skmod.gens_dict))

x2:  {'a010': 1}
x4:  {'a012': 1}
x19:  {'a103': 1}
xi30:  {'A102': -1}


x2*x4

In [319]:
test_rel_backup = {
    A2('xi30')*A2('x9'): 1/q*A2('x9')*A2('xi30'),
    A2('xi8')*A2('x19'): q*A2('x19')*A2('xi8'),
    A2('xi30')*A2('x19'): q*A2('x19')*A2('xi30'),
    A2('x19')*A2('x2'): q*A2('x2')*A2('x19'),
    A2('x19')*A2('x4'): q*A2('x4')*A2('x19'),
    A2('xi30')*A2('x4'): q*A2('x4')*A2('xi30'),
    A2('x3')*A2('x2'): 1/q*A2('x2')*A2('x3')
}

In [214]:
# compare to good cases
fine_variables_1 = [x2, x3, x4, x5, x6, x7, xi1]
for g2,g1 in itertools.combinations(fine_variables_1,2):
    if not g1*g2==g2*g1:
        print(g1*g2,g2*g1)
        
print("\n\n Second Example:\n")
fine_variables_2 = [x3, x6, x8, x11, x21, x26, x29, Mi, Li, xi1, xi4, xi7, xi9, xi13, xi14, xi19, xi20, xi25, xi30, xi33, x3, x6, x13, x21, x23, x33, xi2, xi7, xi11, xi14, xi15, xi19, xi20, xi24, xi29, xi30]
for g2,g1 in itertools.combinations(fine_variables_2,2):
    if not g1*g2==g2*g1:
        print(g1*g2,g2*g1)

q*x2*x3 x2*x3
q*x2*x5 x2*x5
1/q*x2*x6 x2*x6
q*x2*xi1 x2*xi1
1/q*x3*xi1 x3*xi1
q*x4*x5 x4*x5
1/q*x4*x6 x4*x6
q*x5*x6 x5*x6
q*x5*x7 x5*x7
1/q*x5*xi1 x5*xi1
1/q*x6*x7 x6*x7
q*x6*xi1 x6*xi1


 Second Example:

q*x3*Li x3*Li
1/q*x3*xi1 x3*xi1
1/q*x3*xi14 x3*xi14
q*x3*x23 x3*x23
q*x3*xi2 x3*xi2
1/q*x3*xi14 x3*xi14
q*x6*x8 x6*x8
q*x6*x11 x6*x11
1/q*x6*Mi x6*Mi
1/(q^2)*x6*Li x6*Li
q*x6*xi1 x6*xi1
1/q*x6*xi4 x6*xi4
q*x6*xi7 x6*xi7
1/q*x6*xi19 x6*xi19
q*x6*xi25 x6*xi25
1/q*x6*x23 x6*x23
1/q*x6*xi2 x6*xi2
q*x6*xi7 x6*xi7
1/q*x6*xi11 x6*xi11
1/q*x6*xi19 x6*xi19
1/q*x6*xi24 x6*xi24
1/q*x8*x29 x8*x29
q*x8*Li x8*Li
q*x8*xi13 x8*xi13
q*x8*xi19 x8*xi19
1/q*x8*xi20 x8*xi20
x6*x8 q*x6*x8
1/q*x8*x13 x8*x13
q*x8*xi19 x8*xi19
1/q*x8*xi20 x8*xi20
q*x8*xi29 x8*xi29
q*x11*Li x11*Li
q*x11*xi19 x11*xi19
1/q*x11*xi20 x11*xi20
x6*x11 q*x6*x11
1/q*x11*xi15 x11*xi15
q*x11*xi19 x11*xi19
1/q*x11*xi20 x11*xi20
1/q*x21*Li x21*Li
q*x21*xi1 x21*xi1
1/q*x21*x23 x21*x23
1/q*x21*xi2 x21*xi2
1/q*x21*xi15 x21*xi15
q*x26*Li x26

In [302]:
all_relations = H2.relations()
all_relations

{L*M: q*M*L,
 x5*M: 1/q*M*x5,
 x6*M: q*M*x6,
 x13*M: q*M*x13,
 x19*M: q*M*x19,
 x20*M: 1/q*M*x20,
 x29*M: q*M*x29,
 Li*M: 1/q*M*Li,
 xi5*M: q*M*xi5,
 xi6*M: 1/q*M*xi6,
 xi13*M: 1/q*M*xi13,
 xi19*M: 1/q*M*xi19,
 xi20*M: q*M*xi20,
 xi29*M: 1/q*M*xi29,
 x1*L: (q^3)*L*x1,
 x2*L: 1/(q^2)*L*x2,
 x3*L: 1/q*L*x3,
 x4*L: 1/q*L*x4,
 x5*L: 1/q*L*x5,
 x6*L: (q^2)*L*x6,
 x7*L: q*L*x7,
 x8*L: 1/q*L*x8,
 x10*L: q*L*x10,
 x11*L: 1/q*L*x11,
 x13*L: q*L*x13,
 x14*L: q*L*x14,
 x15*L: 1/q*L*x15,
 x16*L: 1/(q^2)*L*x16,
 x17*L: 1/(q^2)*L*x17,
 x18*L: 1/q*L*x18,
 x19*L: q*L*x19,
 x20*L: 1/(q^2)*L*x20,
 x21*L: q*L*x21,
 x22*L: 1/(q^3)*L*x22,
 x23*L: (q^2)*L*x23,
 x24*L: 1/q*L*x24,
 x25*L: q*L*x25,
 x26*L: 1/q*L*x26,
 x27*L: q*L*x27,
 x30*L: 1/(q^2)*L*x30,
 x31*L: 1/(q^2)*L*x31,
 Mi*L: q*L*Mi,
 xi1*L: 1/(q^3)*L*xi1,
 xi2*L: (q^2)*L*xi2,
 xi3*L: q*L*xi3,
 xi4*L: q*L*xi4,
 xi5*L: q*L*xi5,
 xi6*L: 1/(q^2)*L*xi6,
 xi7*L: 1/q*L*xi7,
 xi8*L: q*L*xi8,
 xi10*L: 1/q*L*xi10,
 xi11*L: q*L*xi11,
 xi13*L: 1/q*L*xi13,
 xi14

## back off debugging

In [14]:
# This ideal should be generated by the quantum A-polynomial
# it is infact (1).
# The whole relations ideal isn't (1).
# If I include either the monomial relations or the crossing relations I get (1).
relations_ideal.elimination_ideal([g for name,g in H2.gens_dict().items() if name[0]=='x'])

Left Ideal (1) of Noncommutative Multivariate Polynomial Ring in M, L, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, Mi, Li, xi1, xi2, xi3, xi4, xi5, xi6, xi7, xi8, xi9, xi10, xi11, xi12, xi13, xi14, xi15, xi16, xi17, xi18, xi19, xi20, xi21, xi22, xi23, xi24, xi25, xi26, xi27, xi28, xi29, xi30, xi31, xi32, xi33, xi34 over Fraction Field of Univariate Polynomial Ring in q over Rational Field, nc-relations: {L*M: 1/q*M*L,
 x5*M: q*M*x5,
 x6*M: 1/q*M*x6,
 x13*M: 1/q*M*x13,
 x19*M: 1/q*M*x19,
 x20*M: q*M*x20,
 x29*M: 1/q*M*x29,
 Li*M: q*M*Li,
 xi5*M: 1/q*M*xi5,
 xi6*M: q*M*xi6,
 xi13*M: q*M*xi13,
 xi19*M: q*M*xi19,
 xi20*M: 1/q*M*xi20,
 xi29*M: q*M*xi29,
 x1*L: 1/(q^3)*L*x1,
 x2*L: (q^2)*L*x2,
 x3*L: q*L*x3,
 x4*L: q*L*x4,
 x5*L: q*L*x5,
 x6*L: 1/(q^2)*L*x6,
 x7*L: 1/q*L*x7,
 x8*L: q*L*x8,
 x10*L: 1/q*L*x10,
 x11*L: q*L*x11,
 x13*L: 1/q*L*x13,
 x14*L: 1/q*L*x14,
 x15*L: q*L*x1

## Commutative case

In [126]:
A_comm = FreeAlgebra(QQ,2*dim_wo_q,['M','L'] + ['x{}'.format(i) for i in range(1,dim_wo_q-2+1)]+['Mi','Li']+['xi{}'.format(i) for i in range(1,dim_wo_q-2+1)])
H_comm = A_comm.g_algebra({})

comm_monomial_relations = [get_g_alg_element_from_coord(inv_lattice.basis_matrix()*c,H_comm,qq_overide_value=-1)-1 for c in skmod.monomial_relations]
comm_crossing_rel = [
    reduce(H_comm.summation,
           [-1]+[
               get_g_alg_element_from_coord(inv_lattice.basis_matrix()*mon,H_comm,qq_overide_value=-1)
               for mon in relation]
          ) for relation in skmod.crossing_relations]
comm_inverses_relations = [H_comm(A_comm('M*Mi'))-1,H_comm(A_comm('L*Li'))-1]+[H_comm(A_comm('x{0}*xi{1}'.format(j,j)))-1 for j in range(1,dim_wo_q-2+1)]

comm_relations_ideal = H_comm.ideal(comm_monomial_relations + comm_crossing_rel + comm_inverses_relations)

In [142]:
comm_all_example = H_comm.ideal([H_comm(A_comm('x9*x19^2*x33*xi8*xi30')) - 1, H_comm(A_comm('x2*x3*x4')) - 1])
comm_all_example.std().gens()

(x2*x3*x4 - 1, x9*x19^2*x33*xi8*xi30 - 1)

In [28]:
all_comm_x_variables = [g for name,g in H_comm.gens_dict().items() if name[0]=='x']

In [32]:
comm_relations_ideal.elimination_ideal([x1])

KeyboardInterrupt: 

In [ ]:
comm_relations_ideal.elimination_ideal(all_comm_x_variables)

# David Code

In [ ]:
MLspan = t41.invariant_sublattice.span([t41.meridian,t41.longitude])
dir(t41.invariant_sublattice)
B=[t41.gens_dict['qrt2'],t41.meridian,t41.longitude]
for b in t41.invariant_sublattice.basis():
    if not b in span(B):
        B.append(b)

new_invariant_sublattice=t41.invariant_sublattice.submodule_with_basis(B)
B.remove(t41.gens_dict['qrt2'])

def vector_to_monomial(v):
    assert v in t41.invariant_sublattice
    gens_list=[]
    for i in range(len(v)):
        if v[i]>0:
            gens_list.append(QuantumTorus.gen(i)^v[i])
        if v[i]<0:
            gens_list.append(QuantumTorus.gen(i+len(B))^-v[i])
    return mul(gens_list)

R.<A>=PolynomialRing(QQ)
F=R.fraction_field()
names = ["M","L"] + ['x{}'.format(i) for i in range(len(B)-2)] + ["Mi","Li"] + ['x{}i'.format(i) for i in range(len(B)-2)]
QuantumTorusFA = FreeAlgebra(F,len(names),names)
QuantumTorusFA.inject_variables()
crossrels = {QuantumTorusFA(names[i])*QuantumTorusFA(names[j]): A^((matrix(B[i])*t41.omega_with_q*matrix(B[j]).transpose())[0,0])*QuantumTorusFA(names[j])*QuantumTorusFA(names[i]) for i in range(len(B)) for j in range(len(B)) if i>j} | {QuantumTorusFA(names[i+len(B)])*QuantumTorusFA(names[j+len(B)]): A^((matrix(B[i])*t41.omega_with_q*matrix(B[j]).transpose())[0,0])*QuantumTorusFA(names[j+len(B)])*QuantumTorusFA(names[i+len(B)]) for i in range(len(B)) for j in range(len(B)) if i>j} | {QuantumTorusFA(names[i+len(B)])*QuantumTorusFA(names[j]): A^((matrix(B[j])*t41.omega_with_q*matrix(B[i]).transpose())[0,0])*QuantumTorusFA(names[j])*QuantumTorusFA(names[i+len(B)]) for i in range(len(B)) for j in range(len(B))} 
QuantumTorus=QuantumTorusFA.g_algebra(crossrels)